In [1]:
import os
import pandas as pd
import numpy as np
from sqlalchemy import create_engine,text
import validation_data_input
import plotly.express as px
import toml
import psrc_theme

# to show plotly figures in quarto HTML file
import plotly.io as pio
pio.renderers.default = "plotly_mimetype+notebook_connected"
pio.templates.default = "simple_white+psrc_color" # set plotly template

config = toml.load(os.path.join(os.getcwd(), '..\\..\\..\\..\\configuration', 'validation_configuration.toml'))
input_config = toml.load(os.path.join(os.getcwd(), '..\\..\\..\\..\\configuration', 'input_configuration.toml'))
# model_dir = os.path.join(os.getcwd(), '..\\..\\..\\..\\')

In [2]:
hh = validation_data_input.get_data('household')

In [3]:
# parcel land use data
df_parcel = pd.read_csv(os.path.join(config['model_dir'],r'outputs\landuse\buffered_parcels.txt'),
                        delim_whitespace=True,
                        usecols=['parcelid','emptot_1','hh_1'])

In [4]:
# Load parcel geography lookups
conn = create_engine('sqlite:///'+config['model_dir']+'/inputs/db/'+input_config['db_name'])
parcel_geog = pd.read_sql(text("SELECT ParcelID,CountyName,rg_proposed,district_name FROM " + 'parcel_'+input_config['base_year']+'_geography'), con=conn.connect())


# parcel_urbansim = pd.read_csv(os.path.join(config['model_dir'], 'inputs/scenario/landuse/parcels_urbansim.txt'), delim_whitespace=True, usecols=['parcelid', 'EMPOFC_P','EMPTOT_P'])

In [5]:
# df_parcel_geog = parcel_geog.merge(parcel_urbansim, left_on='ParcelID', right_on='PARCELID', how='left')
df_hh = hh.merge(df_parcel, left_on='hhparcel', right_on='parcelid', how='left'). \
     merge(parcel_geog, left_on='hhparcel', right_on='ParcelID', how='left')


- income, hh density, employment density grouped into very low, low, medium, medium-high and high

In [6]:
# Group income, hh density, and employment density into 4 groups
var_group = df_hh.loc[df_hh['source'] == 'model',['hhincome','emptot_1','hh_1']].quantile([.125, .25, .50, .75])

var_group

,hhincome,emptot_1,hh_1
0.125,30866.0,0.000000,81.955748
0.250,56262.0,8.753380,167.028400
0.500,105870.0,126.912676,357.802998
0.750,180100.0,578.216024,748.890740


In [7]:
# data manipulation
# add income group
df_hh['hhincome_group'] = pd.cut(df_hh['hhincome'],bins=[-9999999.0] + var_group['hhincome'].tolist() + [9999999.0], labels=['very low', 'low', 'medium', 'medium-high', 'high'])
# add hh density groups
df_hh['hh_density_group'] = pd.cut(df_hh['hh_1'],bins=[-9999999.0] + var_group['hh_1'].tolist() + [9999999.0], labels=['very low', 'low', 'medium', 'medium-high', 'high'])
# add employment density groups
df_hh['emp_density_group'] = pd.cut(df_hh['emptot_1'],bins=[-9999999.0] + var_group['emptot_1'].tolist() + [9999999.0], labels=['very low', 'low', 'medium', 'medium-high', 'high'])


In [8]:
def plot_hh_stat(df:pd.DataFrame, var:str, title_cat:str, wid = 700):
    df_plot = df.groupby(['source',var])['hhexpfac'].sum().reset_index()
    df_plot['percentage'] = df_plot.groupby(['source'], group_keys=False)['hhexpfac'].\
            apply(lambda x: x / float(x.sum()))
    
    df_plot_ct = df.groupby(['source',var])['hhexpfac'].count().reset_index(). \
        rename(columns={'hhexpfac':'sample count'})
    df_plot = df_plot.merge(df_plot_ct, on=['source',var])
    
    fig = px.bar(df_plot.sort_values(by=['source']), x=var, y="percentage", color="source",
                 hover_data=['sample count'],
                 barmode="group",title=title_cat)
    fig.update_layout(height=400, width=wid, font=dict(size=11),
                      yaxis=dict(tickformat=".2%"))
    fig.show()

## demographics

In [9]:
plot_hh_stat(df_hh, 'hhincome_group', 'household income')

## Home Location

In [10]:
plot_hh_stat(df_hh, 'CountyName', 'home county')

In [11]:
plot_hh_stat(df_hh, 'district_name', 'home district', wid=900)

In [12]:
plot_hh_stat(df_hh, 'hh_density_group', 'home density')